# SaaS Analytics — Churn, MRR & Feature Adoption

This notebook answers product and revenue questions for a SaaS platform by combining:
- **subscription + churn events** (who churns, when, and why),
- **feature usage** (what users adopt and how intensively),
- **account attributes** (industry, plan tier).

## Analysis goals
1. Establish baseline **churn KPIs** (context).
2. Quantify **feature–retention relationship** (active vs churned users).
3. Break down **adoption by plan tier and industry** (feature–market fit).
4. Measure **subscription breadth / intensity** and its relationship to churn.
5. Export clean CSVs for dashboarding (Power BI).

> Note: This notebook expects processed datasets under `data/processed/` (generated by the data-prep notebook).


In [ ]:
from pathlib import Path
import os

PROJECT_DIR = Path.cwd()
DATA_DIR = Path(os.getenv("DATA_DIR", PROJECT_DIR / "data"))
PROCESSED_DIR = Path(os.getenv("PROCESSED_DIR", DATA_DIR / "processed"))
REFERENCE_DIR = Path(os.getenv("REFERENCE_DIR", DATA_DIR / "reference"))
OUTPUT_DIR = Path(os.getenv("OUTPUT_DIR", PROJECT_DIR / "outputs"))

for d in (PROCESSED_DIR, REFERENCE_DIR, OUTPUT_DIR):
    d.mkdir(parents=True, exist_ok=True)

def require_file(path: Path) -> Path:
    if not path.exists():
        raise FileNotFoundError(
            f"Missing file: {path}\n"
            f"Expected processed data under {PROCESSED_DIR} (run Preparing_Data.ipynb first)."
        )
    return path

def save_output(df, filename: str) -> Path:
    out = OUTPUT_DIR / filename
    df.to_csv(out, index=False)
    return out

def profile_df(df, name: str, n: int = 5) -> None:
    print(f"--- {name} ---")
    print("shape:", df.shape)
    display(df.head(n))


In [ ]:
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt


In [ ]:
# Input paths (portable)
ACCOUNTS_PATH = require_file(PROCESSED_DIR / 'accounts.csv')
CHURN_EVENTS_PATH = require_file(PROCESSED_DIR / 'churn_events.csv')
FEATURE_USAGE_PATH = require_file(PROCESSED_DIR / 'feature_usage.csv')
SUBSCRIPTIONS_PATH = require_file(PROCESSED_DIR / 'subscriptions.csv')
SUPPORT_TICKETS_PATH = require_file(PROCESSED_DIR / 'support_tickets.csv')
PLAN_TIER_DATA_PATH = require_file(REFERENCE_DIR / 'data_about_plan_tier.csv')


## Load datasets

Read processed datasets produced by the data-prep notebook.

In [ ]:
accounts_df = pd.read_csv(ACCOUNTS_PATH)
churn_events_df = pd.read_csv(CHURN_EVENTS_PATH)
feature_usage_df = pd.read_csv(FEATURE_USAGE_PATH)
subscriptions_df = pd.read_csv(SUBSCRIPTIONS_PATH)
plan_tier_data_df = pd.read_csv(PLAN_TIER_DATA_PATH)


## Churn KPIs

Compute basic churn statistics used as context for feature analyses.

In [ ]:
# --- Churn KPIs (baseline context) ---
subs_number = subscriptions_df.shape[0]
churn_subs_number = int(subscriptions_df["churn_flag"].sum())
active_subs_number = subs_number - churn_subs_number
churn_rate = churn_subs_number / subs_number

print(f"Total subscriptions: {subs_number:,}")
print(f"Active subscriptions: {active_subs_number:,}")
print(f"Churned subscriptions: {churn_subs_number:,}")
print(f"Churn rate: {churn_rate:.2%}")


## Feature retention: active vs churn

Compare feature adoption rates across churned vs active subscriptions.

In [ ]:
feature_subscription = pd.merge(feature_usage_df, subscriptions_df, on='subscription_id', how='inner')


In [ ]:
profile_df(feature_subscription, 'feature_subscription (joined feature usage + subscriptions)')


In [ ]:
feature_subscription_account = feature_subscription.merge(accounts_df, on='account_id', how='inner')
profile_df(feature_subscription_account, 'feature_subscription_account (add account attributes)')


In [ ]:
feature_subscription_active = feature_subscription[feature_subscription['churn_flag'] == False]
print('Active rows:', len(feature_subscription_active))


In [ ]:
feature_subscription_churn = feature_subscription[feature_subscription['churn_flag'] == True]
print('Churned rows:', len(feature_subscription_churn))


In [ ]:
feature_usage_active = (feature_subscription_active.groupby("feature_name")["subscription_id"].nunique()/active_subs_number).sort_values(ascending=False).reset_index()
feature_usage_active.columns = ["feature_name", "usage_rate_active"]


In [ ]:
feature_usage_churn = (feature_subscription_churn.groupby("feature_name")["subscription_id"].nunique()/churn_subs_number).sort_values(ascending=False).reset_index()
feature_usage_churn.columns = ["feature_name", "usage_rate_churn"]


In [ ]:
feature_usage_comparison = feature_usage_active.merge(feature_usage_churn, on="feature_name", how="inner")
feature_usage_comparison["retention_lift"] = feature_usage_comparison["usage_rate_active"] - feature_usage_comparison["usage_rate_churn"]
feature_usage_comparison.sort_values("retention_lift", ascending=False)


In [ ]:
out_path = save_output(feature_usage_comparison, "feature_retention_delta_active_vs_churn.csv")
print("Wrote:", out_path)


In [ ]:
feature_usage = (feature_subscription.groupby(["feature_name", "churn_flag"])["subscription_id"].nunique()/active_subs_number).sort_values(ascending=False)
feature_usage.rename("usage_rate", inplace=True)


In [ ]:
subs_per_churn = (
    feature_subscription[["subscription_id", "churn_flag"]]
        .drop_duplicates()
        .groupby("churn_flag")["subscription_id"]
        .nunique()
)

feature_usage_churn = (
    feature_subscription
        .groupby(["feature_name", "churn_flag"])["subscription_id"]
        .nunique()
        .reset_index()
        .merge(subs_per_churn, on="churn_flag")
)

feature_usage_churn["usage_rate"] = (
    feature_usage_churn["subscription_id_x"]
    / feature_usage_churn["subscription_id_y"]
)


## Feature adoption by plan & industry

Break down feature adoption by plan tier and by customer industry.

In [ ]:
feature_usage_per_plan = (feature_subscription.groupby(["feature_name", "plan_tier"])["subscription_id"].nunique()/subs_number).sort_values(ascending=False)
feature_usage_per_plan.rename("usage_rate", inplace=True)


In [ ]:
subs_per_plan = (
    feature_subscription[["subscription_id", "plan_tier"]]
        .drop_duplicates()
        .groupby("plan_tier")["subscription_id"]
        .nunique()
)

feature_usage_per_plan = (
    feature_subscription
        .groupby(["feature_name", "plan_tier"])["subscription_id"]
        .nunique()
        .reset_index()
        .merge(subs_per_plan, on="plan_tier")
)

feature_usage_per_plan["usage_rate"] = (
    feature_usage_per_plan["subscription_id_x"]
    / feature_usage_per_plan["subscription_id_y"]
)


In [ ]:
feature_usage_plan_wide = (
    feature_usage_per_plan
        .pivot(
            index="feature_name",
            columns="plan_tier",
            values="usage_rate"
        )
        .rename(columns={
            "Basic": "usage_rate_basic",
            "Pro": "usage_rate_pro",
            "Enterprise": "usage_rate_enterprise"
        })
        .reset_index()
)


In [ ]:
feature_usage_plan_wide["delta_enterprise_basic"] = feature_usage_plan_wide["usage_rate_enterprise"] - feature_usage_plan_wide["usage_rate_basic"]
feature_usage_plan_wide["delta_pro_basic"] = feature_usage_plan_wide["usage_rate_pro"] - feature_usage_plan_wide["usage_rate_basic"]
feature_usage_plan_wide


In [ ]:
out_path = save_output(feature_usage_plan_wide, "feature_adoption_by_plan.csv")
print("Wrote:", out_path)


In [ ]:
feature_usage_basic = feature_usage_per_plan[feature_usage_per_plan["plan_tier"] == "Basic"]
feature_usage_basic = feature_usage_basic[["feature_name", "usage_rate"]]
feature_usage_basic.columns = ["feature_name", "usage_rate_basic"]


In [ ]:
feature_usage_pro = feature_usage_per_plan[feature_usage_per_plan["plan_tier"] == "Pro"]
feature_usage_pro = feature_usage_pro[["feature_name", "usage_rate"]]
feature_usage_pro.columns = ["feature_name", "usage_rate_pro"]


In [ ]:
feature_usage_enterprise = feature_usage_per_plan[feature_usage_per_plan["plan_tier"] == "Enterprise"]
feature_usage_enterprise = feature_usage_enterprise[["feature_name", "usage_rate"]]
feature_usage_enterprise.columns = ["feature_name", "usage_rate_enterprise"]


In [ ]:
feature_usage_per_plan = feature_usage_basic.merge(feature_usage_pro, on="feature_name", how="inner")
feature_usage_per_plan = feature_usage_per_plan.merge(feature_usage_enterprise, on="feature_name", how="inner")


In [ ]:
feature_usage = (feature_subscription_account.groupby(["feature_name", "industry"])["subscription_id"].nunique()/subs_number).sort_values(ascending=False)
feature_usage.rename("usage_rate", inplace=True)


In [ ]:
subs_per_industry = (
    feature_subscription_account[["subscription_id", "industry"]]
        .drop_duplicates()
        .groupby("industry")["subscription_id"]
        .nunique()
)

feature_usage_industry = (
    feature_subscription_account
        .groupby(["feature_name", "industry"])["subscription_id"]
        .nunique()
        .reset_index()
        .merge(subs_per_industry, on="industry")
)

feature_usage_industry["usage_rate"] = (
    feature_usage_industry["subscription_id_x"]
    / feature_usage_industry["subscription_id_y"]
)


In [ ]:
out_path = save_output(feature_usage, "feature_adoption_rate.csv")
print("Wrote:", out_path)


In [ ]:
feature_subscription_active["usage_date"] = pd.to_datetime(feature_usage_df["usage_date"])
feature_subscription_active["month"] = feature_subscription_active["usage_date"].dt.month
feature_subscription_active["year"] = feature_subscription_active["usage_date"].dt.year
feature_subscription_active


In [ ]:
# usage_month = (feature_usage_df.groupby(["year", "month", "subscription_id", "feature_name"])["usage_count"].sum()).sort_values(ascending=False)
# usage_month
# average_usage_per_month = (usage_month.groupby(["subscription_id", "feature_name"])["usage_count"].mean()).sort_values(ascending=False)
# average_usage_per_month
usage_month = (
    feature_subscription_active
        .groupby(
            ["year", "month", "subscription_id", "feature_name"],
            as_index=False
        )
        .agg(monthly_usage_count=("usage_count", "sum"))
)

average_usage_per_month = (
    usage_month
        .groupby(["subscription_id", "feature_name"], as_index=False)
        .agg(
            avg_monthly_usage=("monthly_usage_count", "mean")
        )
        .sort_values("avg_monthly_usage", ascending=False)
)
average_usage_per_month["avg_monthly_usage"].unique()


In [ ]:
from matplotlib import pyplot as plt
average_usage_per_month['avg_monthly_usage'].plot(kind='hist', bins=20, title='avg_monthly_usage')
plt.gca().spines[['top', 'right',]].set_visible(False)


In [ ]:
not_used_feature = average_usage_per_month[average_usage_per_month['avg_monthly_usage']==0]
print('Features with 0 average monthly usage:', len(not_used_feature))
display(not_used_feature.head(10))


In [ ]:
feature_subscription_churn = feature_subscription[feature_subscription["churn_flag"] == True]


In [ ]:
feature_subscription_churn["usage_date"] = pd.to_datetime(feature_usage_df["usage_date"])
feature_subscription_churn["month"] = feature_subscription_churn["usage_date"].dt.month
feature_subscription_churn["year"] = feature_subscription_churn["usage_date"].dt.year
feature_subscription_churn


In [ ]:
usage_month_churn = (
    feature_subscription_churn
        .groupby(
            ["year", "month", "subscription_id", "feature_name"],
            as_index=False
        )
        .agg(monthly_usage_count=("usage_count", "sum"))
)

average_usage_per_month_churn = (
    usage_month_churn
        .groupby(["subscription_id", "feature_name"], as_index=False)
        .agg(
            avg_monthly_usage=("monthly_usage_count", "mean")
        )
        .sort_values("avg_monthly_usage", ascending=False)
)
average_usage_per_month_churn["avg_monthly_usage"].unique()


In [ ]:
from matplotlib import pyplot as plt
average_usage_per_month_churn['avg_monthly_usage'].plot(kind='hist', bins=20, title='avg_monthly_usage_churn')
plt.gca().spines[['top', 'right',]].set_visible(False)


In [ ]:
not_used_feature_churn = average_usage_per_month_churn[average_usage_per_month_churn['avg_monthly_usage']==0]
print('Churn cohort — features with 0 average monthly usage:', len(not_used_feature_churn))
display(not_used_feature_churn.head(10))


## Feature intensity and subscription breadth

How many features are used per subscription and how usage breadth correlates with churn.

In [ ]:
number_of_features = feature_usage_df['feature_name'].nunique()
print('Number of distinct features:', number_of_features)


In [ ]:
used_features_per_sub_active = (
    feature_subscription_active
    .groupby(["subscription_id"], as_index=False)
    .agg(
        used_features_count=("feature_name", "nunique")
    )
    .assign(
        pct_used_features=lambda x: (x["used_features_count"]/number_of_features).round(2)
    )
    .sort_values("used_features_count", ascending=False)
)
used_features_per_sub_churn = (
    feature_subscription_churn
    .groupby(["subscription_id"], as_index=False)
    .agg(
        used_features_count=("feature_name", "nunique")
    )
    .assign(
        pct_used_features=lambda x: (x["used_features_count"]/number_of_features).round(2)
    )
    .sort_values("used_features_count", ascending=False)
)
print(used_features_per_sub_active)
print(used_features_per_sub_churn)


In [ ]:
avg_used_features_count_active = used_features_per_sub_active["used_features_count"].mean()
avg_used_features_count_churn = used_features_per_sub_churn["used_features_count"].mean()
print(avg_used_features_count_active)
print(avg_used_features_count_churn)


In [ ]:
sub_acc = subscriptions_df.merge(accounts_df, on="account_id", how="inner")
sub_acc = sub_acc[["subscription_id", "industry", "plan_tier_x",  "churn_flag_x"]]
sub_acc.rename(columns={"plan_tier_x": "plan_tier", "churn_flag_x": "churn_flag"}, inplace=True)
sub_acc = sub_acc.drop_duplicates()


In [ ]:
used_features_per_sub_active = used_features_per_sub_active.merge(sub_acc, on="subscription_id", how="inner")
used_features_per_sub_churn = used_features_per_sub_churn.merge(sub_acc, on="subscription_id", how="inner")
subscription_breadth = pd.concat([used_features_per_sub_active, used_features_per_sub_churn], ignore_index=True)


In [ ]:
subscription_breadth_active = subscription_breadth[subscription_breadth["churn_flag"] == False]


In [ ]:
subscription_breadth_churn = subscription_breadth[subscription_breadth["churn_flag"] == True]


In [ ]:
subscription_breadth["churn_flag"] = subscription_breadth["churn_flag"].replace({
    True: "churn",
    False: "active"
})
subscription_breadth.rename(columns={"churn_flag": "subscription_status"}, inplace=True)
subscription_breadth


In [ ]:
out_path = save_output(subscription_breadth, "subscription_breadth.csv")
print("Wrote:", out_path)
out_path = save_output(subscription_breadth_churn, "subscription_breadth_churn.csv")
print("Wrote:", out_path)
out_path = save_output(subscription_breadth_active, "subscription_breadth_active.csv")
print("Wrote:", out_path)


In [ ]:
from matplotlib import pyplot as plt
used_features_per_sub_active['used_features_count'].plot(kind='hist', bins=13, title='feature_usage')
plt.gca().spines[['top', 'right',]].set_visible(False)


In [ ]:
from matplotlib import pyplot as plt
used_features_per_sub_churn['used_features_count'].plot(kind='hist', bins=10, title='feature_usage_churn')
plt.gca().spines[['top', 'right',]].set_visible(False)


In [ ]:
subs_using_feature_active = (
    feature_subscription_active
    .groupby(["feature_name"], as_index=False)
    .agg(
        subs_count_active=("subscription_id", "nunique")
    )
    .assign(
        pct_subs_using_feature_active=lambda x: (x["subs_count_active"]/active_subs_number).round(2)
    )
    .sort_values("subs_count_active", ascending=False)
)
subs_using_feature_churn = (
    feature_subscription_churn
    .groupby(["feature_name"], as_index=False)
    .agg(
        subs_count_churn=("subscription_id", "nunique")
    )
    .assign(
        pct_subs_using_feature_churn=lambda x: (x["subs_count_churn"]/churn_subs_number).round(2)
    )
    .sort_values("subs_count_churn", ascending=False)
)
print(subs_using_feature_active)
print(subs_using_feature_churn)


In [ ]:
feature_subs = subs_using_feature_active.merge(subs_using_feature_churn, on="feature_name", how="inner")
feature_subs["adoption_delta"] = (
    feature_subs["pct_subs_using_feature_active"] - feature_subs["pct_subs_using_feature_churn"]
)

feature_subs.sort_values("adoption_delta", ascending=False)


In [ ]:
subs_using_feature_active_per_plan = (
    feature_subscription_active
    .groupby(["feature_name", "plan_tier"], as_index=False)
    .agg(
        subs_count_active=("subscription_id", "nunique")
    )
    .assign(
        pct_subs_using_feature_active=lambda x: (x["subs_count_active"]/active_subs_number).round(2)
    )
    .sort_values("subs_count_active", ascending=False)
)
subs_using_feature_churn_per_plan = (
    feature_subscription_churn
    .groupby(["feature_name", "plan_tier"], as_index=False)
    .agg(
        subs_count_churn=("subscription_id", "nunique")
    )
    .assign(
        pct_subs_using_feature_churn=lambda x: (x["subs_count_churn"]/churn_subs_number).round(2)
    )
    .sort_values("subs_count_churn", ascending=False)
)
print(subs_using_feature_active_per_plan)
print(subs_using_feature_churn_per_plan)


In [ ]:
feature_subs_per_plan = subs_using_feature_active_per_plan.merge(subs_using_feature_churn_per_plan, on=["feature_name", "plan_tier"], how="inner")
feature_subs_per_plan["adoption_delta"] = (
    feature_subs_per_plan["pct_subs_using_feature_active"] - feature_subs_per_plan["pct_subs_using_feature_churn"]
)

feature_subs_per_plan.sort_values("adoption_delta", ascending=False)


In [ ]:
feature_subs_per_plan_dashboard = feature_subs_per_plan[
    (feature_subs_per_plan["subs_count_active"] >= 100) &
    (feature_subs_per_plan["subs_count_churn"] >= 20)
]
feature_subs_per_plan_dashboard = feature_subs_per_plan_dashboard[["feature_name", "plan_tier", "pct_subs_using_feature_active", "pct_subs_using_feature_churn", "adoption_delta"]]
feature_subs_per_plan_dashboard.sort_values("adoption_delta", ascending=False)


In [ ]:
print(subs_using_feature_active['feature_name'].nunique())
print(subs_using_feature_churn['feature_name'].nunique())


In [ ]:
from matplotlib import pyplot as plt
subs_using_feature_active['subs_count'].plot(kind='hist', bins=13, title='subs_using_feature')
plt.gca().spines[['top', 'right',]].set_visible(False)


In [ ]:
from matplotlib import pyplot as plt
subs_using_feature_churn['subs_count'].plot(kind='hist', bins=11, title='subs_using_feature_churn')
plt.gca().spines[['top', 'right',]].set_visible(False)


In [ ]:
features = feature_usage_df["feature_name"].unique()


In [ ]:
sub_feature_active = feature_subscription_active[["subscription_id", "feature_name"]]
sub_feature_churn = feature_subscription_churn[["subscription_id", "feature_name"]]


In [ ]:
print(sub_feature_active)
print(sub_feature_churn)


In [ ]:
subscription_accounts = subscriptions_df.merge(accounts_df, on="account_id", how="inner")


In [ ]:
subscription_accounts = subscription_accounts[["subscription_id", "account_id", "industry", "plan_tier_x", "seats_x", "mrr_amount", "arr_amount", "churn_flag_x"]]


In [ ]:
total_mrr = subscription_accounts.loc[
    subscription_accounts["churn_flag_x"] == False,
    "mrr_amount"
    ].sum()
arps = total_mrr/subscription_accounts.loc[
    subscription_accounts["churn_flag_x"] == False,
    "seats_x"
].sum()
print(total_mrr)
print(arps)


In [ ]:
avg_monthly_revenue_per_industry = (
    subscription_accounts
    .groupby(["industry"], as_index=False)
    .agg(
        avg_monthly_revenue=("mrr_amount", "mean")
    )
    .sort_values("avg_monthly_revenue", ascending=False)
  )


## Additional cuts and dashboards

Supplementary slices used for dashboarding / further exploration.

In [ ]:
monthly_revenue_per_seat_per_industry = (
    subscription_accounts
    .groupby(["industry"], as_index=False)
    .agg(
        seats_per_industry=("seats_x", "sum"),
        monthly_revenue=("mrr_amount", "sum")
    )
    .sort_values("monthly_revenue", ascending=False)
  )
monthly_revenue_per_seat_per_industry["monthly_revenue_per_seat"] = monthly_revenue_per_seat_per_industry["monthly_revenue"]/monthly_revenue_per_seat_per_industry["seats_per_industry"]
monthly_revenue_per_seat_per_industry.sort_values("monthly_revenue_per_seat", ascending=False)


In [ ]:
churn_rate_per_industry = (
    subscription_accounts
    .groupby(["industry"], as_index=False)
    .agg(
        number_of_subs_per_industry=("subscription_id", "nunique"),
        number_of_churned_subs_per_industry=("churn_flag_x", "sum")
    )
    .assign(
        churn_rate_per_industry=lambda x: (x["number_of_churned_subs_per_industry"]/x["number_of_subs_per_industry"]).round(2)
        )
    )
churn_rate_per_industry.sort_values("churn_rate_per_industry", ascending=False)


In [ ]:
industry_unit_economics = churn_rate_per_industry.merge(monthly_revenue_per_seat_per_industry, on="industry", how="inner")


In [ ]:
out_path = save_output(industry_unit_economics, "industry_unit_economics.csv")
print("Wrote:", out_path)


In [ ]:
feature_subscription_active_per_industry = feature_subscription_active.merge(accounts_df, on="account_id", how="inner")
feature_subscription_churn_per_industry = feature_subscription_churn.merge(accounts_df, on="account_id", how="inner")
print(feature_subscription_active_per_industry)
print(feature_subscription_churn_per_industry)


In [ ]:
number_of_subs_per_industry = (
    churn_rate_per_industry
    .assign(
        number_of_active_subs_per_industry=lambda x:
            x["number_of_subs_per_industry"] - x["number_of_churned_subs_per_industry"]
    )[["industry", "number_of_active_subs_per_industry"]]
)
number_of_subs_per_industry_active = number_of_subs_per_industry[["industry", "number_of_active_subs_per_industry"]]
number_of_subs_per_industry_churn = churn_rate_per_industry[["industry", "number_of_churned_subs_per_industry"]]
print(number_of_subs_per_industry_active)
print(number_of_subs_per_industry_churn)


In [ ]:
feature_subscription_active_per_industry = feature_subscription_active_per_industry.merge(number_of_subs_per_industry_active, on="industry", how="inner")
feature_subscription_churn_per_industry = feature_subscription_churn_per_industry.merge(number_of_subs_per_industry_churn, on="industry", how="inner")
print(feature_subscription_active_per_industry)
print(feature_subscription_churn_per_industry)


In [ ]:
subs_using_feature_active_per_industry = (
    feature_subscription_active_per_industry
    .groupby(["industry", "feature_name"], as_index=False)
    .agg(
        subs_count_active=("subscription_id", "nunique"),
        number_of_active_subs_per_industry=("number_of_active_subs_per_industry", "first")
    )
    .assign(
        pct_subs_using_feature_active=lambda x: (x["subs_count_active"]/x["number_of_active_subs_per_industry"]).round(4)
    )
    .sort_values("subs_count_active", ascending=False)
)
subs_using_feature_churn_per_industry = (
    feature_subscription_churn_per_industry
    .groupby(["industry", "feature_name"], as_index=False)
    .agg(
        subs_count_churn=("subscription_id", "nunique"),
        number_of_churn_subs_per_industry=("number_of_churned_subs_per_industry", "first")
    )
    .assign(
        pct_subs_using_feature_churn=lambda x: (x["subs_count_churn"]/x["number_of_churn_subs_per_industry"]).round(4)
    )
    .sort_values("subs_count_churn", ascending=False)
)
print(subs_using_feature_active_per_industry)
print(subs_using_feature_churn_per_industry)


In [ ]:
feature_subs_per_industry = subs_using_feature_active_per_industry.merge(subs_using_feature_churn_per_industry, on=["industry", "feature_name"], how="inner", validate="one_to_one")
feature_subs_per_industry["adoption_delta"] = (
    feature_subs_per_industry["pct_subs_using_feature_active"] - feature_subs_per_industry["pct_subs_using_feature_churn"]
)
feature_subs_per_industry["weighted_adoption_delta"] = feature_subs_per_industry["adoption_delta"] * np.log(feature_subs_per_industry["subs_count_churn"] + 1)

feature_subs_per_industry.sort_values("adoption_delta", ascending=False).head(40)


In [ ]:
feature_subs_per_industry_dashboard = feature_subs_per_industry[["industry", "feature_name", "pct_subs_using_feature_active", "pct_subs_using_feature_churn", "adoption_delta", "weighted_adoption_delta"]]
feature_subs_per_industry_dashboard.sort_values("weighted_adoption_delta", ascending=False)


In [ ]:
out_path = save_output(feature_subs_per_industry_dashboard, "feature_industry_weighted_delta.csv")
print("Wrote:", out_path)


In [ ]:
subscriptions_df


In [ ]:
churn_events_df


In [ ]:
accounts_df


In [ ]:
churn_subscriptions = churn_events_df.merge(accounts_df, on="account_id", how="inner")


In [ ]:
churns_per_plan = (
    churn_subscriptions
    .groupby(["plan_tier"], as_index=False)
    .agg(
        number_of_churns_per_plan=("churn_event_id", "nunique")
    )
    )


In [ ]:
churn_driver_per_plan = (
    churn_subscriptions
    .groupby(["plan_tier", "reason_code"], as_index=False)
    .agg(
        number_of_churns=("churn_event_id", "nunique")
    )
    .sort_values("number_of_churns", ascending=False)
  )


In [ ]:
churn_driver_per_plan = churn_driver_per_plan.merge(churns_per_plan, on="plan_tier", how="inner")


In [ ]:
churn_driver_per_plan["churn_driver_pct"] = (churn_driver_per_plan["number_of_churns"]/churn_driver_per_plan["number_of_churns_per_plan"]).round(2)
churn_driver_per_plan


In [ ]:
out_path = save_output(churn_driver_per_plan, "churn_reasons_by_plan.csv")
print("Wrote:", out_path)


In [ ]:
support_tickets_df = pd.read_csv(SUPPORT_TICKETS_PATH)


In [ ]:
support_account = support_tickets_df.merge(accounts_df, on="account_id", how="inner")


In [ ]:
plan_tier_support = (
    support_account
    .groupby(["plan_tier", "priority"], as_index=False)
    .agg(
        number_of_tickets=("ticket_id", "nunique"),
        resolution_time_hours=("resolution_time_hours", "mean"),
        first_response_time_minutes=("first_response_time_minutes", "mean"),
        satisfaction_score=("satisfaction_score", "mean")
    )
  )


In [ ]:
out_path = save_output(plan_tier_support, "support_by_plan.csv")
print("Wrote:", out_path)


In [ ]:
plan_tier_support_no_priority = (
    support_account
    .groupby(["plan_tier"], as_index=False)
    .agg(
        number_of_tickets=("ticket_id", "nunique"),
        resolution_time_hours=("resolution_time_hours", "mean"),
        first_response_time_minutes=("first_response_time_minutes", "mean"),
        satisfaction_score=("satisfaction_score", "mean")
    )
  )


In [ ]:
print(active_subs_number)
print(total_mrr)
print(churn_rate)
print(arps)


In [ ]:
main_kpi_df = pd.DataFrame([{
    "active_subs_number": active_subs_number,
    "total_mrr": total_mrr,
    "churn_rate": churn_rate,
    "arps": arps
}])


In [ ]:
out_path = save_output(main_kpi_df, "main_kpi.csv")
print("Wrote:", out_path)


## Exports

All CSV exports are written to `outputs/`.